In [1]:
from ppm_benchmark.core.benchmark_loader import BenchmarkLoader


loader = BenchmarkLoader()
benchmark = loader.load_from_config('../benchmark_configs/remote/mini/remote_attribute_suffix_classification.yaml', max_workers=2)

tasks = benchmark.get_tasks()
tasks

In [2]:
import numpy as np
from ppm_benchmark.core.benchmark_loader import BenchmarkLoader
from ppm_benchmark.utils.label_encoder import PPMLabelEncoder
from tqdm.notebook import tqdm

loader = BenchmarkLoader()
benchmark = loader.load_from_folder('attribute_suffix_classification')
tasks = benchmark.get_tasks()
results = dict()

for task_name in tqdm(tasks):
    task = benchmark.load_task(task_name)
    train = task.get_train_data()
    test = task.get_test_data()
    string_cols = train.select_dtypes(include=['object']).columns
    encoder = PPMLabelEncoder()
    train[string_cols] = encoder.fit_transform(train[string_cols])
    string_cols = test.select_dtypes(include=['object']).columns
    test[string_cols] = encoder.transform_with_new_labels(test[string_cols])
    X = train.drop('target', axis=1)
    y = train['target'].apply(lambda x: np.array(x))
    
    np.random.shuffle(test['target'])
    
    results[task.name] = test['target']

In [3]:
import pickle

with open('attribute_suffix_classification/test_results.pkl', 'wb') as f:
    pickle.dump(results, f)

In [1]:
import pickle
from ppm_benchmark.core.benchmark_loader import BenchmarkLoader


loader = BenchmarkLoader()
benchmark = loader.load_from_folder('attribute_suffix_classification')
evaluator = benchmark.get_evaluator()

with open('attribute_suffix_classification/test_results.pkl', 'rb') as f:
    results = pickle.load(f)

In [2]:
for task_name, result in results.items():
    task = benchmark.load_task(task_name)
    test = task.get_test_data().drop('target', axis=1)
    evaluator.add_predictions(task_name, result, 'shuffle')

In [ ]:
evaluator.evaluate()

In [ ]:
evaluator.plot_by_fraction_completed('DamerauLevenshtein')